In [1]:
#import relevant libraries
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [2]:
#set data_location containing profiles.csv file
data_location = "data/profiles.csv"

In [3]:
#import profiles.csv file as a dataframe
all_data = pd.read_csv(data_location)
print(len(all_data))

59946


In [4]:
# set max_columns and max_colwidth to make it easier to see data in DataFrame
pd.options.display.max_columns = 60
pd.options.display.max_colwidth = 100
all_data.head()

,age,body_type,diet,drinks,drugs,education,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,ethnicity,height,income,job,last_online,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,22,a little extra,strictly anything,socially,never,working on college/university,about me:<br />\n<br />\ni would love to think that i was some some kind of intellectual:\neithe...,"currently working as an international agent for a freight\nforwarding company. import, export, d...",making people laugh.<br />\nranting about a good salting.<br />\nfinding simplicity in complexit...,"the way i look. i am a six foot half asian, half caucasian mutt. it\nmakes it tough not to notic...","books:<br />\nabsurdistan, the republic, of mice and men (only book that made me\nwant to cry), ...",food.<br />\nwater.<br />\ncell phone.<br />\nshelter.,duality and humorous things,trying to find someone to hang out with. i am down for anything\nexcept a club.,i am new to california and looking for someone to wisper my secrets\nto.,you want to be swept off your feet!<br />\nyou are tired of the norm.<br />\nyou want to catch a...,"asian, white",75.0,-1,transportation,2012-06-28-20-30,"south san francisco, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
1,35,average,mostly other,often,sometimes,working on space camp,i am a chef: this is what that means.<br />\n1. i am a workaholic.<br />\n2. i love to cook rega...,dedicating everyday to being an unbelievable badass.,being silly. having ridiculous amonts of fun wherever. being a\nsmart ass. ohh and i can cook. ;),NaN,i am die hard christopher moore fan. i don't really watch a lot of\ntv unless there is humor inv...,delicious porkness in all of its glories.<br />\nmy big ass doughboy's sinking into 15 new inche...,NaN,NaN,i am very open and will share just about anything.,NaN,white,70.0,80000,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (poorly)",single
2,38,thin,anything,socially,NaN,graduated from masters program,"i'm not ashamed of much, but writing public text on an online\ndating site makes me pleasantly u...","i make nerdy software for musicians, artists, and experimenters to\nindulge in their own weirdne...",improvising in different contexts. alternating between being\npresent and decidedly outside of a...,my large jaw and large glasses are the physical things people\ncomment on the most. when suffici...,"okay this is where the cultural matrix gets so specific, it's like\nbeing in the crosshairs.<br ...",movement<br />\nconversation<br />\ncreation<br />\ncontemplation<br />\ntouch<br />\nhumor,NaN,viewing. listening. dancing. talking. drinking. performing.,"when i was five years old, i was known as ""the boogerman"".","you are bright, open, intense, silly, ironic, critical, caring,\ngenerous, looking for an explor...",NaN,68.0,-1,NaN,2012-06-27-09-10,"san francisco, california",NaN,straight,has cats,NaN,m,pisces but it doesn&rsquo;t matter,no,"english, french, c++",available
3,23,thin,vegetarian,socially,NaN,working on college/university,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books according to the library\nof congress classification s...,socially awkward but i do my best,"bataille, celine, beckett. . .<br />\nlynch, jarmusch, r.w. fassbender. . .<br />\ntwin peaks &a...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.,white,71.0,20000,student,2012-06-28-14-22,"berkeley, california",doesn&rsquo;t want kids,straight,likes cats,NaN,m,pisces,no,"english, german (poorly)",single
4,29,athletic,NaN,socially,never,graduated from college/university,"hey how's it going? curre

In [5]:
all_data.columns.values

array(['age', 'body_type', 'diet', 'drinks', 'drugs', 'education',
       'essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5',
       'essay6', 'essay7', 'essay8', 'essay9', 'ethnicity', 'height',
       'income', 'job', 'last_online', 'location', 'offspring',
       'orientation', 'pets', 'religion', 'sex', 'sign', 'smokes',
       'speaks', 'status'], dtype=object)

In [6]:
#drop columns that will not be used for project
columns_to_drop = [
    'education',
    'ethnicity',
    'job',
    'last_online',
    'location',
    'offspring',
    'orientation',
    'pets',
    'religion',
    'sign',
    'speaks',
    'status',
    'income'
]
all_data.drop(columns_to_drop, axis=1, inplace=True)
all_data.columns.values

array(['age', 'body_type', 'diet', 'drinks', 'drugs', 'essay0', 'essay1',
       'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7',
       'essay8', 'essay9', 'height', 'sex', 'smokes'], dtype=object)

In [7]:
#check which columns have NaN values
all_data.isna().any()

age          False
body_type     True
diet          True
drinks        True
drugs         True
essay0        True
essay1        True
essay2        True
essay3        True
essay4        True
essay5        True
essay6        True
essay7        True
essay8        True
essay9        True
height        True
sex          False
smokes        True
dtype: bool

In [8]:
#remove NaNs in essay columns
essay_columns_with_nan_values = {
    'essay0': '',
    'essay1': '',
    'essay2': '',
    'essay3': '',
    'essay4': '',
    'essay5': '',
    'essay6': '',
    'essay7': '',
    'essay8': '',
    'essay9': ''
}
all_data.fillna(essay_columns_with_nan_values, inplace = True)

In [9]:
#remove NaNs in other columns
other_columns_with_nan_values = [
    'body_type', 
    'diet', 
    'drinks', 
    'drugs', 
    'height', 
    'smokes'
]
all_data.dropna(subset=other_columns_with_nan_values, inplace=True)

In [10]:
all_data.isna().any()

age          False
body_type    False
diet         False
drinks       False
drugs        False
essay0       False
essay1       False
essay2       False
essay3       False
essay4       False
essay5       False
essay6       False
essay7       False
essay8       False
essay9       False
height       False
sex          False
smokes       False
dtype: bool

In [11]:
# determine value counts for diet responses
all_data['diet'].value_counts()

mostly anything        12290
anything                4357
strictly anything       3341
mostly vegetarian       2322
mostly other             741
strictly vegetarian      632
vegetarian               366
strictly other           310
other                    233
mostly vegan             222
strictly vegan           160
vegan                     86
mostly kosher             64
mostly halal              36
strictly halal            15
strictly kosher           14
halal                      7
kosher                     6
Name: diet, dtype: int64

In [17]:
# map diet responses to a numeric code:
# 0 for vegan
# 1 for vegetarian
# 2 for religious
# 3 for any other response

def map_diet_to_code(diet):
    if "vegan" in diet:
        return 0
    elif "vegetarian" in diet:
        return 1
    elif "kosher" in diet or "halal" in diet:
        return 2
    else:
        return 3
    
all_data['diet_code'] = all_data['diet'].apply(map_diet_to_code)

#check that mapping diet responses has worked 

all_data['diet_code'].value_counts()

2    21414
1     3320
0      468
Name: diet_code, dtype: int64